In [1]:
import os
import cv2
import numpy as np
from glob import glob
import random

def get_images(folder):
    """Return a list of all .jpg images in the given folder."""
    return glob(os.path.join(folder, '*.jpg'))

def calculate_global_avg_and_std_rgb(image_paths):
    """
    Calculate the global per-channel average and standard deviation 
    over all images in the folder.
    Returns:
      avg_rgb: (r_avg, g_avg, b_avg)
      std_rgb: (r_std, g_std, b_std) -- with zero std replaced by 1.
    """
    r_values, g_values, b_values = [], [], []
    
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            continue
        # Split channels in BGR order.
        b, g, r = cv2.split(img)
        r_values.append(np.mean(r))
        g_values.append(np.mean(g))
        b_values.append(np.mean(b))
    
    if not r_values or not g_values or not b_values:
        return (np.nan, np.nan, np.nan), (np.nan, np.nan, np.nan)
    
    # Compute global averages for each channel.
    r_avg = np.mean(r_values)
    g_avg = np.mean(g_values)
    b_avg = np.mean(b_values)
    
    # Compute global standard deviations for each channel.
    r_std = np.std(r_values)
    g_std = np.std(g_values)
    b_std = np.std(b_values)
    
    # Prevent division by zero
    r_std = r_std if r_std != 0 else 1
    g_std = g_std if g_std != 0 else 1
    b_std = b_std if b_std != 0 else 1
    
    return (r_avg, g_avg, b_avg), (r_std, g_std, b_std)

def adjust_image_color(img, global_avg, global_std):
    """
    Apply z-normalization to the image:
      - For each channel, subtract the global average and divide by the global std.
    """
    b, g, r = cv2.split(img)
    
    r_adjusted = (r - global_avg[0]) / global_std[0]
    g_adjusted = (g - global_avg[1]) / global_std[1]
    b_adjusted = (b - global_avg[2]) / global_std[2]
    
    adjusted_img = cv2.merge([b_adjusted, g_adjusted, r_adjusted])
    return adjusted_img

def process_images(image_paths, global_avg, global_std, output_folder):
    """
    Process each image:
      - Normalize it using the global average and std.
      - Print a random pixel's normalized RGB value.
      - Save the normalized image as a compressed .npz file.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            continue
        
        adjusted_img = adjust_image_color(img, global_avg, global_std)
        
        # Pick a random pixel to print its normalized RGB values.
        height, width, _ = adjusted_img.shape
        rand_x = random.randint(0, width - 1)
        rand_y = random.randint(0, height - 1)
        random_pixel_rgb = adjusted_img[rand_y, rand_x]
        print(f"Random pixel at ({rand_x}, {rand_y}) in {os.path.basename(img_path)}: {random_pixel_rgb}")
        
        # Save the normalized image as a compressed .npz file.
        npz_filename = os.path.basename(img_path).replace('.jpg', '.npz')
        npz_file_path = os.path.join(output_folder, npz_filename)
        np.savez_compressed(npz_file_path, image=adjusted_img)

def main(input_folder, output_folder):
    image_paths = get_images(input_folder)
    
    # Compute global average and std over all images.
    global_avg, global_std = calculate_global_avg_and_std_rgb(image_paths)
    print("Global average (R, G, B):", global_avg)
    print("Global std (R, G, B):", global_std)
    
    process_images(image_paths, global_avg, global_std, output_folder)

if __name__ == "__main__":
    input_folder = r'D:\Download\meta\Cropped and Deblurred'
    output_folder = r'D:\Download\meta\Normalized'
    main(input_folder, output_folder)


Global average (R, G, B): (np.float64(119.78383463270524), np.float64(114.35775092770072), np.float64(102.01629815220294))
Global std (R, G, B): (np.float64(36.56271047434325), np.float64(31.985386499252215), np.float64(37.592576222642556))
Random pixel at (484, 166) in sa_1.jpg: [-0.50585249  1.23938628  0.47086677]
Random pixel at (331, 175) in sa_1000.jpg: [ 0.4783844  -1.0116417  -1.03339808]
Random pixel at (106, 325) in sa_10002.jpg: [-1.99550831 -2.69991269 -1.47100239]
Random pixel at (142, 436) in sa_10006.jpg: [2.02124221 2.08352177 1.78367973]
Random pixel at (336, 96) in sa_10007.jpg: [0.90400034 2.20857888 2.49478674]
Random pixel at (134, 472) in sa_10010.jpg: [ 0.29217742 -0.19877049 -0.40434187]
Random pixel at (458, 286) in sa_10011.jpg: [-2.42112426 -3.20014113 -3.22141967]
Random pixel at (152, 293) in sa_10012.jpg: [-1.27728139 -2.1371557  -2.20945968]
Random pixel at (467, 279) in sa_10014.jpg: [ 0.07936944 -0.51141326 -0.84194619]
Random pixel at (235, 168) in sa_